# LGBM - Training

Sources
1. [LGBM & Deberta Explained by ZULQARNAIN ALI](https://www.kaggle.com/code/zulqarnainalipk/lgbm-deberta-explained)

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb
import pickle as pkl
import wandb
from lightgbm import log_evaluation, early_stopping
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.paths import Paths
from lib.utils.utils import seed_everything
from lib.criterion.metrics import log_metrics
from lib.model.utils import quadratic_weighted_kappa, qwk_obj, get_score

In [4]:
seed_everything()

### Setting Up WandB

In [5]:
WANDB_PROJECT = "Kaggle_ASE_2.0"
WANDB_NAME = f"LGBM-ASE-4"

In [6]:
wandb.login(key=os.environ.get('WANDB_API_KEY'))
wandb.init(
    project=WANDB_PROJECT,
    name=WANDB_NAME,
    save_code=True,
    job_type="train",
    config=config,
)
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shakleenishfar (laplacesdemon43). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ishfar/.netrc


## Model Training

### Dataset Creation

In [7]:
df = pd.read_csv(Paths.FEATURE_ENGG_CSV_PATH)
df.shape

(17307, 211)

In [8]:
ids = df.loc[:, "essay_id"]
X, y = df.drop(columns=["score", "essay_id"]), df.loc[:, "score"]
del df

X.shape, y.shape

((17307, 209), (17307,))

### Callbacks

In [9]:
callbacks = [
    log_evaluation(period=25),
    early_stopping(stopping_rounds=75, first_metric_only=True),
]

### Constants

In [10]:
a = config.lgbm_a
b = config.lgbm_b

### Training Loop

In [11]:
def train_loop(X_train, y_train, X_valid, y_valid):
    y_train = y_train.astype(np.float32) - a
    y_valid = y_valid.astype(np.float32) - a

    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics="None",
        learning_rate=0.01,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.3,
        reg_alpha=0.7,
        reg_lambda=0.1,
        n_estimators=700,
        random_state=config.random_seed,
        extra_trees=True,
        class_weight="balanced",
        n_jobs=6,
        verbosity=0,
    )

    # Train model
    predictor = model.fit(
        X_train,
        y_train,
        eval_names=["train", "valid"],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks,
    )

    return model, predictor

In [12]:
def validate_model(idx, X_valid, y_valid, id_valid, predictor):
    y_valid_int = y_valid.astype(np.int64).to_numpy()
    preds = predictor.predict(X_valid) + a
    preds = preds.clip(1, 6).round().astype(np.int64)

    oof = pd.DataFrame(
        {
            "essay_id": id_valid,
            "score": y_valid_int - 1,
            "pred_score": preds - 1,
        }
    )
    log_metrics(oof, f"Fold {idx}")

    return oof

### Fold Loop

In [13]:
skf = StratifiedKFold(
    n_splits=config.lgbm_n_folds,
    random_state=config.random_seed,
    shuffle=True,
)

In [14]:
oof_df = pd.DataFrame()

for idx, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_valid, y_valid = X.loc[valid_idx], y.loc[valid_idx]
    id_valid = ids.loc[valid_idx].to_numpy()

    model, predictor = train_loop(X_train, y_train, X_valid, y_valid)

    with open(f"output/LGBM/{idx}.pkl", "wb") as file:
        pkl.dump({"predictor": predictor, "model": model}, file)

    oof = validate_model(idx, X_valid, y_valid, id_valid, predictor)
    oof_df = pd.concat([oof_df, oof])

[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.443479	valid's QWK: 0.478976
[50]	train's QWK: 0.738798	valid's QWK: 0.756205
[75]	train's QWK: 0.801221	valid's QWK: 0.815883
[100]	train's QWK: 0.835834	valid's QWK: 0.843054
[125]	train's QWK: 0.859141	valid's QWK: 0.865977
[150]	train's QWK: 0.868265	valid's QWK: 0.876946
[175]	train's QWK: 0.872425	valid's QWK: 0.880006
[200]	train's QWK: 0.875181	valid's QWK: 0.883233
[225]	train's QWK: 0.877504	valid's QWK: 0.887912
[250]	train's QWK: 0.879589	valid's QWK: 0.891068
[275]	train's QWK: 0.881953	valid's QWK: 0.891194
[300]	train's QWK: 0.882925	valid's QWK: 0.893239
[325]	train's QWK: 0.884014	valid's QWK: 0.893665
[350]	train's QWK: 0.884783	valid's QWK: 0.893362
[375]	train's QWK: 0.885235	valid's QWK: 0.893769
[400]	train's QWK: 0.885804	valid's QWK: 0.893313
Early stopping, best iteration is:
[330]	train's QWK: 0.884013	valid's QWK: 0.894407
E

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.445974	valid's QWK: 0.402015
[50]	train's QWK: 0.736091	valid's QWK: 0.727993
[75]	train's QWK: 0.802692	valid's QWK: 0.790698
[100]	train's QWK: 0.838353	valid's QWK: 0.830327
[125]	train's QWK: 0.859473	valid's QWK: 0.84886
[150]	train's QWK: 0.869247	valid's QWK: 0.853369
[175]	train's QWK: 0.872628	valid's QWK: 0.860017
[200]	train's QWK: 0.876106	valid's QWK: 0.863314
[225]	train's QWK: 0.879267	valid's QWK: 0.866426
[250]	train's QWK: 0.881527	valid's QWK: 0.870534
[275]	train's QWK: 0.88286	valid's QWK: 0.869972
[300]	train's QWK: 0.88413	valid's QWK: 0.870491
[325]	train's QWK: 0.884804	valid's QWK: 0.870101
[350]	train's QWK: 0.885	valid's QWK: 0.871176
[375]	train's QWK: 0.885803	valid's QWK: 0.87245
[400]	train's QWK: 0.886419	valid's QWK: 0.873373
[425]	train's QWK: 0.886761	valid's QWK: 0.873373
[450]	train's QWK: 0.887053	valid's QWK: 0.

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:36: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.440956	valid's QWK: 0.450025
[50]	train's QWK: 0.73887	valid's QWK: 0.729784
[75]	train's QWK: 0.800432	valid's QWK: 0.783951
[100]	train's QWK: 0.835144	valid's QWK: 0.823484
[125]	train's QWK: 0.858387	valid's QWK: 0.850166
[150]	train's QWK: 0.86897	valid's QWK: 0.861721
[175]	train's QWK: 0.87369	valid's QWK: 0.86143
[200]	train's QWK: 0.876226	valid's QWK: 0.86489
[225]	train's QWK: 0.878904	valid's QWK: 0.866617
[250]	train's QWK: 0.880646	valid's QWK: 0.871584
[275]	train's QWK: 0.882524	valid's QWK: 0.872895
[300]	train's QWK: 0.883564	valid's QWK: 0.873255
[325]	train's QWK: 0.884663	valid's QWK: 0.877129
[350]	train's QWK: 0.885212	valid's QWK: 0.878696
[375]	train's QWK: 0.885797	valid's QWK: 0.879447
[400]	train's QWK: 0.885959	valid's QWK: 0.8823
[425]	train's QWK: 0.886323	valid's QWK: 0.882379
[450]	train's QWK: 0.886687	valid's QWK: 0.

In [15]:
score = get_score(oof_df["score"], oof_df["pred_score"])
print(f'Overall Score: {score:<.4f}')
wandb.log({"CV/qwk_score": score})

Overall Score: 0.8854


In [16]:
wandb.log({"oof_table": wandb.Table(dataframe=oof_df)})

In [17]:
log_metrics(
    pd.DataFrame(
        {
            "score": oof_df.score.to_numpy(),
            "pred_score": oof_df.pred_score.to_numpy(),
        }
    ),
    "Overall",
)

In [18]:
wandb.finish()

CV/qwk_score,▁
CV/qwk_score,0.88536
